In [4]:
# Importando Bibliotecas:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV

In [5]:
# Importando o dataframe:
df = pd.read_csv('df_final.csv')
df.columns

Index(['Unnamed: 0', 'titulo', 'preco', 'quartos', 'suites', 'banheiros',
       'vagas', 'privativos', 'link', 'lat', 'long', 'numero_rua', 'bairro',
       'cidade', 'piscina', 'gas_central', 'elevador', 'salao', 'academia'],
      dtype='object')

In [6]:
# Excluindo colunas desnecessárias para o modelo:
df.drop(['Unnamed: 0','titulo','link','lat','long','numero_rua','cidade'], axis = 1, inplace=True)
df

,preco,quartos,suites,banheiros,vagas,privativos,bairro,piscina,gas_central,elevador,salao,academia
0,880000.00,3,1,2,2,79,Fazenda,1,1,1,1,1
1,490000.00,2,1,2,1,69,São João,1,0,1,1,1
2,255000.00,2,0,1,1,49,Itaipava,1,0,0,1,0
3,420000.00,3,0,1,1,72,Centro,0,0,0,1,0
4,250000.00,2,0,1,1,54,Carvalho,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
265,2400000.00,3,3,5,3,155,Centro,1,0,1,1,1
266,755000.00,2,1,1,1,55,Praia Brava,1,1,1,1,1
267,3056529.06,3,3,4,2,193,Fazendinha,1,0,1,1,1
268,2713440.00,3,3,4,2,126,Praia Brava,1,0,1,1,0


In [7]:
# Separando os dados em treino - teste:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['preco'], axis = 1), df['preco'], test_size = 0.30)

print(f'O dataset de treino ficou com dimensões {X_train.shape} enquanto o nosso target de treino ficou {y_train.shape}.')
print(f'O dataset de test ficou com dimensões {X_test.shape} enquanto o nosso target de teste ficou {y_test.shape}.')

O dataset de treino ficou com dimensões (189, 11) enquanto o nosso target de treino ficou (189,).
O dataset de test ficou com dimensões (81, 11) enquanto o nosso target de teste ficou (81,).


In [8]:
# Aplicando Target Encoder na variável 'bairro':

target_encoder = ce.TargetEncoder()
X_train_encoded = target_encoder.fit_transform(X_train[['bairro']], y_train)
X_train_encoded = X_train_encoded.join(X_train.drop('bairro', axis = 1))

# Aplicando MinMaxScaler nas variáveis numéricas:

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)

# Aplicando Target Encoder no Test:
X_test_encoded = target_encoder.transform(X_test[['bairro']])
X_test_encoded = X_test_encoded.join(X_test.drop('bairro', axis = 1))

# Aplicando MinMaxScaler no Test:
X_test_scaled = scaler.transform(X_test_encoded)



In [9]:
# Função para testar vários modelos:
def modelos_and_metricas(X_train, X_test, y_train, y_test):
    # Modelos:
    modelos = {
        'Regressão Linear Múltipla':LinearRegression(),
        'Regressão Ridge': Ridge(),
        'Regressão Lasso': Lasso(),
        'SVR':SVR(),
        'Random Forest Regressor': RandomForestRegressor(),
        'K-Nearest Neighbors': KNeighborsRegressor(),
        'XGBoost Regressor': XGBRegressor()
    }
    resultados = []
    # Resultados de cada modelo:
    for nome_modelo, modelo in modelos.items():
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        mae = mean_absolute_error(y_test,y_pred)
        mse = mean_squared_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = r2_score(y_test,y_pred)

        resultado = {
            'Modelo': nome_modelo,
            'MAE': mae,
            'MSE': mse,
            'MAPE': mape,
            'R-squared':r2
        }
        # Salvando os resultados
        resultados.append(resultado)
    resultados_df = pd.DataFrame(resultados)
    return resultados_df

resultados = modelos_and_metricas(X_train_scaled, X_test_scaled, y_train, y_test)
resultados

,Modelo,MAE,MSE,MAPE,R-squared
0,Regressão Linear Múltipla,284134.301057,1.807324e+11,0.274044,0.742496
1,Regressão Ridge,286617.035256,1.869220e+11,0.261333,0.733677
2,Regressão Lasso,284132.254803,1.807328e+11,0.274037,0.742495
3,SVR,600674.115534,8.418041e+11,0.555160,-0.199387
4,Random Forest Regressor,183933.119795,1.154941e+11,0.154313,0.835446
5,K-Nearest Neighbors,272243.478148,2.152120e+11,0.211999,0.693370
6,XGBoost Regressor,196532.697616,1.133328e+11,0.159523,0.838526


In [10]:
# Instanciando o melhor modelo:
rf = RandomForestRegressor()
# Hiperparâmetros para testar:
param = {'max_depth':[4,5,6,7], 'n_estimators':[20,40,60,80,100]}
# Cross-Validation
cv_rf = GridSearchCV(rf, param_grid = param, scoring = 'neg_mean_absolute_error', n_jobs = None,
                       refit = True, cv = 4, verbose = 1, pre_dispatch = None, error_score = np.nan,
                       return_train_score = True)

# Treinando a Validação Cruzada:
cv_rf.fit(X_train_scaled,y_train)



Fitting 4 folds for each of 20 candidates, totalling 80 fits


GridSearchCV(cv=4, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [4, 5, 6, 7],
                         'n_estimators': [20, 40, 60, 80, 100]},
             pre_dispatch=None, return_train_score=True,
             scoring='neg_mean_absolute_error', verbose=1)

In [11]:
# Buscando os melhores parâmetros da Validação Cruzada:
cv_rf_results = pd.DataFrame(cv_rf.cv_results_)
print(f'Os melhores parâmetros para a Random Forest são: {cv_rf.best_params_}.')


Os melhores parâmetros para a Random Forest são: {'max_depth': 7, 'n_estimators': 20}.


In [12]:
# Utilizando o modelo tunado:
rf_final = RandomForestRegressor(max_depth = 6, n_estimators = 60)
rf_final.fit(X_train_scaled, y_train)
y_pred = rf_final.predict(X_test_scaled)

In [13]:
# Obtendo as métricas da aplicação do modelo:
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test,y_pred)

print('Resultados do Random Forest Regressor pós Cross Validation')
print(f'MAE : {mae}.')
print(f'MSE : {mse}.')
print(f'MAPE : {mape}.')
print(f'R-Squared : {r2}.')


Resultados do Random Forest Regressor pós Cross Validation
MAE : 198390.60197785945.
MSE : 123038253186.02348.
MAPE : 0.16767063195362056.
R-Squared : 0.8246973860335136.


In [84]:
# Lista de Bairros:

bairros = df.bairro.unique(sorted=True).tolist()

TypeError: unique() takes 1 positional argument but 2 were given

In [82]:
# Utilizando o pickle para exportar o modelo:

joblib.dump(rf_final,'model.pkl')
joblib.dump(target_encoder,'encoder.pkl')
joblib.dump(scaler,'scaler.pkl')
joblib.dump(bairros,'bairros.pkl')

# Para Carregar o modelo:
# modelo = joblib.load('model.pkl')

['bairros.pkl']

In [15]:
# # Aplicando Target Encoder no Test:
# X_new_encoded = target_encoder.transform(X_new[['bairro']])
# X_new_encoded = X_new_encoded.join(X_new.drop('bairro', axis = 1))

# # Aplicando MinMaxScaler no Test:
# X_new_scaled = scaler.transform(X_new_encoded)

# X_new_scaled

In [16]:
# y_new = rf_final.predict(X_new_scaled)

In [17]:
# y_new